# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
weather_df = pd.read_csv("output_data/cities.csv")

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Darhan,49.4867,105.9228,53.10,79,99,12.21,MN,1632189785
1,New Norfolk,-42.7826,147.0587,49.50,58,99,9.19,AU,1632189785
2,Solano,38.2333,-121.9511,86.18,18,1,3.44,US,1632189786
3,Busselton,-33.6500,115.3333,61.66,53,1,12.06,AU,1632189786
4,Asău,46.4333,26.4000,45.52,91,96,0.60,RO,1632189787


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df['Humidity'].astype(int)

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Narrowing the weather dataframe to include the following parameters:
    # A max temperature lower than 80 degrees but higher than 70.
    # Wind speed less than 10 mph.
    # Zero cloudiness.

perfect_weather_df = weather_df[(weather_df['Max Temp'] < 80) & (weather_df['Max Temp'] > 70) & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0)]
perfect_weather_df.head(40)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,Yinchuan,38.4681,106.2731,76.32,19,0,6.33,CN,1632189824
159,Riyadh,24.6877,46.7219,75.34,18,0,6.42,SA,1632189820
195,Zhoucheng,35.9122,116.3117,74.32,68,0,5.79,CN,1632189889
264,Dörtyol,36.8616,36.2289,71.26,78,0,2.55,TR,1632189930
275,Ankang,32.6800,109.0172,73.04,71,0,2.46,CN,1632189936
283,Qurayyat,31.3318,37.3428,73.92,43,0,5.70,SA,1632189940
365,Abu Samrah,35.3029,37.1841,73.24,46,0,5.19,SY,1632189986
376,Kiryat Gat,31.6100,34.7642,74.08,76,0,4.43,IL,1632189992
497,Natal,-5.7950,-35.2094,75.85,83,0,6.91,BR,1632190056
506,Lichuan,30.3000,108.8500,72.14,48,0,1.12,CN,1632190061


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
filter_city =perfect_weather_df['City'].tolist()
filter_lat = perfect_weather_df['Lat'].tolist()
filter_lng = perfect_weather_df['Lng'].tolist()
filter_country = perfect_weather_df['Country'].tolist()

hotel_dict = {'City': filter_city, 'Country': filter_country, 'Lat': filter_lat, 'Lng': filter_lng}
hotel_df = pd.DataFrame.from_dict(hotel_df)

hotel_df['Hotel Name'] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
74,Yinchuan,CN,38.4681,106.2731,,,
159,Riyadh,SA,24.6877,46.7219,,,
195,Zhoucheng,CN,35.9122,116.3117,,,
264,Dörtyol,TR,36.8616,36.2289,,,
275,Ankang,CN,32.6800,109.0172,,,
283,Qurayyat,SA,31.3318,37.3428,,,
365,Abu Samrah,SY,35.3029,37.1841,,,
376,Kiryat Gat,IL,31.6100,34.7642,,,
497,Natal,BR,-5.7950,-35.2094,,,
506,Lichuan,CN,30.3000,108.8500,,,


In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
